In [13]:
import numpy as np
import pandas as pd
import re
import unicodedata
from sklearn import linear_model
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn import svm

In [14]:
df_train=pd.read_csv("/content/F2_Claim_Check_Worthiness_train.csv")

In [15]:
df_train

,tweet,label,id
0,"India's gift of 100,000 COVID-19 vaccines arri...",0,0
1,As part of the ongoing nationwide vaccination ...,0,1
2,"Pleased to receive 50,000 doses of Covid-19 va...",0,2
3,Four former presidents have banded together fo...,0,3
4,WSJ: All three of Russia's main intelligence s...,1,4
...,...,...,...
2117,"Some pearl of Wisdom by your fuckeer ""Bill Gat...",1,2117
2118,Top infectious disease expert Dr. Anthony Fauc...,1,2118
2119,Everyone is looking at Bill Gates for the Coro...,1,2119
2120,The president’s first instinct in response to ...,0,2120


In [16]:
df_test=pd.read_csv("/content/F2_Claim_Check_Worthiness_test.csv")

In [17]:
df_test

,tweet,id
0,Native American communities have been hit hard...,0
1,ALERT: CVS Pharmacy is now offering COVID-19 v...,1
2,#COVID19 vaccines are an important tool to hel...,2
3,Pfizer reports that IRL the vaccine is 97% eff...,3
4,Vaccines are complex medicines. Europeans can ...,4
...,...,...
190,"Today, University of Pittsburgh School of Medi...",190
191,UK: Coronavirus vaccine could be ready by Sept...,191
192,Top NIH official Anthony Fauci says coronaviru...,192
193,my flintstones gummies from 15 years ago fight...,193


In [18]:
def clean_text(text):
  text = re.sub('\S*@\S*\s?', '', text)
  text = re.sub('\s+', ' ', text) 
  text = re.sub("\'", '', text)
  text = re.sub('[^a-zA-Z]', ' ', text)
  text = re.sub('\s+[a-zA-Z]\s+^I', ' ', text)
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  text = re.sub(r'\s+', ' ', text)
  text = re.sub(r'^\s*|\s\s*', ' ', text).strip()
  text = text.lower()
  text = word_tokenize(text)
  return text

In [19]:
df_train['tweet'] = df_train['tweet'].apply(lambda x : clean_text(x)).values.tolist()
df_test['tweet'] = df_test['tweet'].apply(lambda x : clean_text(x)).values.tolist()

In [20]:
df_train

,tweet,label,id
0,"[indias, gift, of, covid, vaccines, arrived, b...",0,0
1,"[as, part, of, the, ongoing, nationwide, vacci...",0,1
2,"[pleased, to, receive, doses, of, covid, vacci...",0,2
3,"[four, former, presidents, have, banded, toget...",0,3
4,"[wsj, all, three, of, russias, main, intellige...",1,4
...,...,...,...
2117,"[some, pearl, of, wisdom, by, your, fuckeer, b...",1,2117
2118,"[top, infectious, disease, expert, dr, anthony...",1,2118
2119,"[everyone, is, looking, at, bill, gates, for, ...",1,2119
2120,"[the, president, s, first, instinct, in, respo...",0,2120


Model 1: TF-IDF

In [9]:
nltk.download('punkt')
nltk.download('stopwords')
stopword=set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
#tf-idf
model1=TfidfVectorizer(stop_words=stopword,max_features=10) 

sent_list=df_train['tweet']
train_matrix=model1.fit_transform(sent_list)
X_train=train_matrix
test_list=df_test['tweet']
test_matrix=model1.fit_transform(test_list)
X_test=test_matrix
Y_train=np.array(df_train["label"])

regr =svm.SVC()
regr.fit(X_train, Y_train)
pred_test=regr.predict(X_test)

In [11]:
data = {'label': df_test["id"],
        'id': pred_test}

df_out = pd.DataFrame(data)
# df_out.set_index('label', inplace=True)
df_out.to_csv('mdg1_svm.csv')

In [12]:
df_out.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   label   195 non-null    int64
 1   id      195 non-null    int64
dtypes: int64(2)
memory usage: 3.2 KB


Model2: word2vec+svm

In [26]:
sentences=[]
for i in range(len(df_train)):
  sentences.append(df_train.iloc[i].tweet)
for i in range(len(df_test)):
  sentences.append(df_test.iloc[i].tweet)
print(len(sentences))
print(sentences[0])

2317
['indias', 'gift', 'of', 'covid', 'vaccines', 'arrived', 'barbados', 'earlier', 'today', 'this', 'was', 'a', 'very', 'special', 'moment', 'for', 'all', 'barbadians', 'and', 'i', 'want', 'to', 'thank', 'prime', 'minister', 'modi', 'for', 'his', 'quick', 'decisive', 'and', 'magnanimous', 'action', 'in', 'allowing', 'us', 'to', 'be', 'the', 'beneficiary', 'of', 'these', 'vaccines', 'https', 't', 'co', 'cscb', 'c', 'mt']


In [41]:
import gensim
w2v_model=gensim.models.Word2Vec(sentences=sentences,size=10,window=10,min_count=1)

In [44]:
def w2v_vectors(df):
  va=[]
  for i in range(len(df)):
    v=np.zeros(10)
    l=df.iloc[i].tweet
    for j in l:
      v += w2v_model.wv[j]
    va.append(v)
  return va

In [45]:
df_train_vec=w2v_vectors(df_train)
df_test_vec=w2v_vectors(df_test)

In [46]:
len(df_train_vec)


2122

In [47]:
regr2 =svm.SVC()
regr2.fit(df_train_vec, Y_train)
pred_test=regr.predict(df_test_vec)

In [50]:
data = {'label': pred_test,
        'id': df_test["id"]}

df_out = pd.DataFrame(data)
df_out.set_index('label', inplace=True)
df_out.to_csv('w2v_svm.csv')